In [1]:
pip install flask flask-cors pandas numpy scikit-learn joblib

Note: you may need to restart the kernel to use updated packages.


In [2]:
# app.py
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import numpy as np
import joblib

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS to allow frontend requests

# Load the trained model, scaler, and label encoder
model = joblib.load('best_yield_model.pkl')
scaler = joblib.load('scaler.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Define the features expected by the model
condition_features = ['Temperature', 'Rainfall', 'Humidity', 'Soil_pH', 'Nitrogen', 'Phosphorus', 'Potassium']

# Route for predicting crop yield
@app.route('/predict_yield', methods=['POST'])
def predict_yield():
    try:
        # Get the data from the frontend (JSON format)
        data = request.get_json()

        # Extract crop name and convert to encoded value
        crop_name = data.get('crop', '').strip().lower()
        all_crops_lower = [c.lower() for c in label_encoder.classes_]
        if crop_name not in all_crops_lower:
            return jsonify({'error': 'Crop not found. Please try again.'}), 400
        crop_encoded = label_encoder.transform([label_encoder.classes_[all_crops_lower.index(crop_name)]])[0]

        # Extract condition features
        input_data = {feature: float(data.get(feature.lower(), 0)) for feature in condition_features}
        input_df = pd.DataFrame([input_data], columns=condition_features)

        # Scale the input features
        scaled_input = scaler.transform(input_df)

        # Combine scaled features with the encoded crop
        combined_input = np.hstack([scaled_input, [[crop_encoded]]])

        # Make prediction
        predicted_yield = model.predict(combined_input)[0]

        # Return the result as JSON
        return jsonify({
            'crop': crop_name.title(),
            'predicted_yield': round(predicted_yield, 2),
            'unit': 'tons/ha'
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Route for recommending the best crop
@app.route('/recommend_crop', methods=['POST'])
def recommend_crop():
    try:
        # Get the data from the frontend (JSON format)
        data = request.get_json()

        # Extract condition features
        input_data = {feature: float(data.get(feature.lower(), 0)) for feature in condition_features}
        input_df = pd.DataFrame([input_data], columns=condition_features)

        # Scale the input features
        scaled_input = scaler.transform(input_df)

        # Recommend the best crop by testing all crops
        all_crops = label_encoder.classes_
        crop_predictions = []
        for crop in label_encoder.transform(all_crops):
            combined_input = np.hstack([scaled_input, [[crop]]])
            predicted_yield = model.predict(combined_input)[0]
            crop_predictions.append((crop, predicted_yield))

        # Find the crop with the highest predicted yield
        best_crop_encoded = max(crop_predictions, key=lambda x: x[1])[0]
        best_crop = label_encoder.inverse_transform([best_crop_encoded])[0]

        # Return the result as JSON
        return jsonify({
            'recommended_crop': best_crop
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

FileNotFoundError: [Errno 2] No such file or directory: 'best_yield_model.pkl'